

# Predicting patient perception / how they feel towards themselves(their anxiety/pain ..) from:

## the other EQ-5D-5L scores and sociodemographic features 


In [1]:
import savReaderWriter as spss
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.style as style
style.use('seaborn')

import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interactive, Layout, HTML

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import classification_report 
#from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


from imblearn.over_sampling import SMOTE
import itertools

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [6]:
import savReaderWriter as spss
import pandas as pd


raw_data = spss.SavReader('p29_data.sav', returnHeader = True)
raw_data_list = list(raw_data)
df = pd.DataFrame(raw_data_list)
with spss.SavReader('p29_data.sav', ioUtf8 = True) as reader:
    df = pd.DataFrame(reader.all(), columns = [s.decode('CP1252') for s in reader.header])
        
    
#prom= df[[ 'WEIGHT', 'GENDER', 'AGER','AGE','E1', 'E2', 'E3', 'E4', 'E5', 'E6']].copy()
eq5d= df[[ 'WEIGHT', 'GENDER', 'AGE','E1', 'E2', 'E3', 'E4', 'E5', 'E6']].copy()
eq5d.columns = ['WEIGHT', 'GENDER', 'AGE','MOBILITY', 'SELF CARE', 'USUAL ACTIVITY', 'PAIN', 'ANXIETY', 'HEALTH SCALE']


eq= eq5d[['MOBILITY', 'SELF CARE', 'USUAL ACTIVITY', 'PAIN', 'ANXIETY']].copy()
new = eq5d.copy()
new['GENERAL HEALTH'] = eq.sum(axis=1)
#eq5d.head(3)    

In [ ]:
dff= df[[ 'WEIGHT', 'GENDER', 'AGE','OCC', 'HHCMP5', 'MARITAL','EDU3','INC4','E1', 'E2', 'E3', 'E4', 'E5', 'E6']].copy()




dff.columns = ['WEIGHT', 'GENDER', 'AGE',
               'OCCUPATION','FAMILY_NUMBER','MARITAL','EDUCATION','INCOME',
               'MOBILITY', 'SELF CARE', 'USUAL ACTIVITY', 'PAIN', 'ANXIETY', 'HEALTH SCALE',
               ]

dff['GENERAL HEALTH'] = eq.sum(axis=1)

In [62]:
dff

,WEIGHT,GENDER,AGE,OCCUPATION,FAMILY_NUMBER,MARITAL,EDUCATION,INCOME,MOBILITY,SELF CARE,USUAL ACTIVITY,PAIN,ANXIETY,HEALTH SCALE,GENERAL HEALTH
0,0.955652,1.0,37.0,3.0,2.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,80.0,5.0
1,1.012784,2.0,25.0,2.0,2.0,2.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,81.0,7.0
2,0.997378,1.0,47.0,5.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,82.0,5.0
3,1.029702,2.0,19.0,5.0,3.0,1.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,82.0,8.0
4,0.991771,1.0,37.0,5.0,1.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,25.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4507,0.834959,2.0,48.0,5.0,4.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0,67.0,6.0
4508,0.777193,1.0,42.0,4.0,5.0,3.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,61.0,8.0
4509,1.009954,1.0,33.0,3.0,4.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,93.0,6.0
4510,0.903712,1.0,40.0,3.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,3.0,70.0,7.0


In [3]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    """
    This function prints and plots the confusion matrix.`.
    """

    #style.use('default')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [4]:
from collections import Counter
from matplotlib import pyplot

## labels is highly imbalanced

In [7]:

Label= widgets.Dropdown(options=['MOBILITY', 'SELF CARE','USUAL ACTIVITY', 'PAIN', 'ANXIETY'], value='ANXIETY')

@interact(Label = Label)
def pltt(Label):

    y = new[Label]
    x = new


    counter = Counter(y)
    for k,v in counter.items():
        per = v / len(y) * 100
        print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
    # plot the distribution
    pyplot.bar(counter.keys(), counter.values())
    pyplot.show()

interactive(children=(Dropdown(description='Label', index=4, options=('MOBILITY', 'SELF CARE', 'USUAL ACTIVITY…

In [15]:

Label= widgets.Dropdown(options=['MOBILITY', 'SELF CARE','USUAL ACTIVITY', 'PAIN', 'ANXIETY'], value='ANXIETY')

@interact(Label = Label)
def random_forrest(Label):
    x = new.drop(Label,axis = 1)
    y = new[Label]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42,shuffle = True)

    random_forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    clf = random_forest.fit(x_train,y_train)

    y_pred = clf.predict(x_test)
    
    
    
    print ("Number of training observations ; ", len(y_train))
    print ("________________________")
    print ("Number of test observations ; ", len(y_test))
    print ("________________________")

    print ("cross Testing result========")
    scores = cross_val_score(clf, x_test, y_test, cv=5)
    print (scores)
    print ("________________________")
    print ("Testing result mean ========")
    print (scores.mean())
    print ("________________________")


    cnf_matrix = confusion_matrix(y_test,y_pred)
    print ("Printing confusion Matrix ========")
    plot_confusion_matrix(cnf_matrix,classes=[1,2,3,4,5])
    print ("________________________")

interactive(children=(Dropdown(description='Label', index=4, options=('MOBILITY', 'SELF CARE', 'USUAL ACTIVITY…

## Results after oversampling (SMOTE) 

In [10]:
oversample = SMOTE()

In [63]:

Label= widgets.Dropdown(options=['MOBILITY', 'SELF CARE','USUAL ACTIVITY', 'PAIN', 'ANXIETY'], value='ANXIETY')

@interact(Label = Label)
def pltt(Label):



    y = new[Label]
    x = new


    oversample = SMOTE()
    x_new, y_new = oversample.fit_resample(x, y)
    # summarize distribution
    counter = Counter(y_new)
    for k,v in counter.items():
        per = v / len(y_new) * 100
        print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
    # plot the distribution
    pyplot.bar(counter.keys(), counter.values())
    pyplot.show()

interactive(children=(Dropdown(description='Label', index=7, options=('WEIGHT', 'GENDER', 'AGE', 'MOBILITY', '…

In [65]:

Label= widgets.Dropdown(options=['MOBILITY', 'SELF CARE','USUAL ACTIVITY', 'PAIN', 'ANXIETY'], value='PAIN')

@interact(Label = Label)
def random_forrest(Label):
    x = new.drop(Label,axis = 1)
    y = new[Label]
    
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.4, random_state=42,shuffle = True)
    x_new, y_new = oversample.fit_resample(x_train, y_train)

    random_forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    clf = random_forest.fit(x_new, y_new)

    y_pred = clf.predict(x_test)
    
    
    print ("Number of training observations ; ", len(y_new))
    print ("________________________")
    print ("Number of test observations ; ", len(y_test))
    print ("________________________")
        
    print ("cross Testing result========")
    scores = cross_val_score(clf, x_test, y_test, cv=5)
    print (scores)
    print ("________________________")
    print ("Testing result mean ========")
    print (scores.mean())
    print ("________________________")


    cnf_matrix = confusion_matrix(y_test,y_pred)
    print ("Printing confusion Matrix ========")
    plot_confusion_matrix(cnf_matrix,classes=[1,2,3,4,5])
    print ("________________________")

interactive(children=(Dropdown(description='Label', index=3, options=('MOBILITY', 'SELF CARE', 'USUAL ACTIVITY…